In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import cifar10

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
%%capture
!pip install wandb

In [3]:
import wandb
from wandb.keras import WandbCallback

wandb.login()

wandb: Currently logged in as: dbhadore (use `wandb login --relogin` to force relogin)


True

In [4]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train/255.
x_test = x_test/255.

CLASS_NAMES = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

print('Shape of x_train: ', x_train.shape)
print('Shape of y_train: ', y_train.shape)
print('Shape of x_test: ', x_test.shape)
print('Shape of y_test: ', y_test.shape)

170500096/170498071 [==============================] - 19s 0us/step
Shape of x_train:  (50000, 32, 32, 3)
Shape of y_train:  (50000, 1)
Shape of x_test:  (10000, 32, 32, 3)
Shape of y_test:  (10000, 1)


In [5]:
def Model():
    inputs = keras.layers.Input(shape=(32, 32, 3))

    x = keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu')(inputs)
    x = keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu')(x)
    x = keras.layers.MaxPooling2D(pool_size=2)(x)

    x = keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu')(x)
    x = keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu')(x)

    x = keras.layers.GlobalAveragePooling2D()(x)

    x = keras.layers.Dense(128, activation='relu')(x)
    x = keras.layers.Dense(32, activation='relu')(x)

    outputs = keras.layers.Dense(len(CLASS_NAMES), activation='softmax')(x)

    return keras.models.Model(inputs=inputs, outputs=outputs)

In [6]:
def train():
    # Specify the hyperparameter to be tuned along with
    # an initial value
    config_defaults = {
        'batch_size': 8,
        'learning_rate': 0.01
    }

    # Initialize wandb with a sample project name
    wandb.init(config=config_defaults)

    # Specify the other hyperparameters to the configuration, if any
    wandb.config.epochs = 25

    # Prepare trainloader
    trainloader = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    trainloader = trainloader.shuffle(1024).batch(wandb.config.batch_size).prefetch(tf.data.experimental.AUTOTUNE)
    # prepare testloader 
    testloader = tf.data.Dataset.from_tensor_slices((x_test, y_test))
    testloader = testloader.batch(wandb.config.batch_size).prefetch(tf.data.experimental.AUTOTUNE)

    # Iniialize model with hyperparameters
    keras.backend.clear_session()
    model = Model()
    
    # Compile the model
    opt = tf.keras.optimizers.Adam(learning_rate=wandb.config.learning_rate) # optimizer with different learning rate specified by config
    model.compile(opt, 'sparse_categorical_crossentropy', metrics=['acc'])
    
    # Train the model
    _ = model.fit(trainloader,
                  epochs=wandb.config.epochs, 
                  validation_data=testloader,
                  callbacks=[WandbCallback()]) # WandbCallback to automatically track metrics
                            
    # Evaluate    
    loss, accuracy = model.evaluate(testloader, callbacks=[WandbCallback()])
    print('Test Error Rate: ', round((1-accuracy)*100, 2))
    wandb.log({'Test Error Rate': round((1-accuracy)*100, 2)}) # wandb.log to track custom metrics

In [7]:
sweep_config = {
  'method': 'bayes', 
  'metric': {
      'name': 'val_loss',
      'goal': 'minimize'
  },
  'early_terminate':{
      'type': 'hyperband',
      'min_iter': 5
  },
  'parameters': {
      'batch_size': {
          'values': [8, 16, 32, 64, 128, 256]
      },
      'learning_rate':{
          'values': [0.01, 0.005, 0.001, 0.0005, 0.0001]
      }
  }
}

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="my-keras-integration-dhiman")
wandb.agent(sweep_id, function=train)

Create sweep with ID: 480xscrq
Sweep URL: https://wandb.ai/dbhadore/my-keras-integration-dhiman/sweeps/480xscrq


wandb: Agent Starting Run: sslcaa6z with config:
wandb: 	batch_size: 8
wandb: 	learning_rate: 0.001


Epoch 1/25
6250/6250 [==============================] - 77s 12ms/step - loss: 1.7001 - acc: 0.3570 - val_loss: 1.4078 - val_acc: 0.4766
Epoch 2/25
6250/6250 [==============================] - 81s 13ms/step - loss: 1.3327 - acc: 0.5152 - val_loss: 1.2846 - val_acc: 0.5330
Epoch 3/25
6250/6250 [==============================] - 83s 13ms/step - loss: 1.1682 - acc: 0.5757 - val_loss: 1.1445 - val_acc: 0.5888
Epoch 4/25
6250/6250 [==============================] - 90s 14ms/step - loss: 1.0624 - acc: 0.6176 - val_loss: 1.0620 - val_acc: 0.6128
Epoch 5/25
6250/6250 [==============================] - 88s 14ms/step - loss: 0.9842 - acc: 0.6486 - val_loss: 0.9933 - val_acc: 0.6429
Epoch 6/25
6250/6250 [==============================] - 89s 14ms/step - loss: 0.9244 - acc: 0.6711 - val_loss: 0.9703 - val_acc: 0.6590
Epoch 7/25
6250/6250 [==============================] - 89s 14ms/step - loss: 0.8784 - acc: 0.6886 - val_loss: 1.1025 - val_acc: 0.6275
Epoch 8/25
6250/6250 [==========================

epoch,24
loss,0.56775
acc,0.80138
val_loss,0.82863
val_acc,0.7267
_runtime,4640
_timestamp,1622275923
_step,25
best_val_loss,0.80147
best_epoch,18
Test Error Rate,27.33


epoch,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
loss,█▆▅▄▄▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
acc,▁▃▄▅▆▆▆▆▇▇▇▇▇▇▇▇▇████████
val_loss,█▇▅▄▃▃▄▂▂▂▂▁▁▂▁▂▂▁▁▁▁▂▁▂▁
val_acc,▁▃▄▅▆▆▅▇▇▇▇▇██▇▇▇████████
_runtime,▁▁▁▁▂▂▂▂▂▂▂▂▃▃▇▇▇▇▇▇▇█████
_timestamp,▁▁▁▁▂▂▂▂▂▂▂▂▃▃▇▇▇▇▇▇▇█████
_step,▁▁▂▂▂▂▃▃▃▄▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
Test Error Rate,▁


...

Epoch 1/25
3125/3125 [==============================] - 79s 25ms/step - loss: 2.3046 - acc: 0.0966 - val_loss: 2.3050 - val_acc: 0.1000
Epoch 2/25
3125/3125 [==============================] - 78s 25ms/step - loss: 2.3046 - acc: 0.0984 - val_loss: 2.3045 - val_acc: 0.1000
Epoch 3/25
3125/3125 [==============================] - 78s 25ms/step - loss: 2.3048 - acc: 0.0967 - val_loss: 2.3034 - val_acc: 0.1000
Epoch 4/25
3125/3125 [==============================] - 78s 25ms/step - loss: 2.3045 - acc: 0.1003 - val_loss: 2.3045 - val_acc: 0.1000
Epoch 5/25
3125/3125 [==============================] - 78s 25ms/step - loss: 2.3046 - acc: 0.1012 - val_loss: 2.3058 - val_acc: 0.1000
Epoch 6/25
3125/3125 [==============================] - 78s 25ms/step - loss: 2.3048 - acc: 0.0977 - val_loss: 2.3038 - val_acc: 0.1000
Epoch 7/25
3125/3125 [==============================] - 78s 25ms/step - loss: 2.3046 - acc: 0.0977 - val_loss: 2.3052 - val_acc: 0.1000
Epoch 8/25
3125/3125 [==========================

epoch,7
loss,2.30456
acc,0.09874
val_loss,2.30563
val_acc,0.1
_runtime,636
_timestamp,1622276994
_step,7
best_val_loss,2.30336
best_epoch,2


epoch,▁▂▃▄▅▆▇█
loss,▄▄▇▁▄█▄▃
acc,▁▄▁▇█▃▃▄
val_loss,▆▄▁▄█▂▆▇
val_acc,▁▁▁▁▁▁▁▁
_runtime,▁▂▃▄▅▆▇█
_timestamp,▁▂▃▄▅▆▇█
_step,▁▂▃▄▅▆▇█


wandb: Agent Starting Run: 2qmyu2x8 with config:
wandb: 	batch_size: 8
wandb: 	learning_rate: 0.01


Epoch 1/25
6250/6250 [==============================] - 85s 14ms/step - loss: 2.3055 - acc: 0.0966 - val_loss: 2.3070 - val_acc: 0.1000
Epoch 2/25
6250/6250 [==============================] - 85s 14ms/step - loss: 2.3052 - acc: 0.0989 - val_loss: 2.3063 - val_acc: 0.1000
Epoch 3/25
6250/6250 [==============================] - 10885s 2s/step - loss: 2.3052 - acc: 0.0989 - val_loss: 2.3055 - val_acc: 0.1000
Epoch 4/25
6250/6250 [==============================] - 8101s 1s/step - loss: 2.3054 - acc: 0.0958 - val_loss: 2.3059 - val_acc: 0.1000
Epoch 5/25
 855/6250 [===>..........................] - ETA: 1:06 - loss: 2.3062 - acc: 0.0944